# メールマーケティングの効果の検証

RCTを行ったデータとバイアスのあるデータを集計してグループ間を比較する。

## データの準備

メールマーケティングの効果をMineThatData E-mail Analytics And Data Mining Challenge datasetというデータセットで分析する。

In [1]:
library("tidyverse")

Warning message:
“package ‘tidyverse’ was built under R version 3.5.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

Warning message:
“package ‘tibble’ was built under R version 3.5.2”
Warning message:
“package ‘tidyr’ was built under R version 3.5.2”
Warning message:
“package ‘purrr’ was built under R version 3.5.2”
Warning message:
“package ‘dplyr’ was built under R version 3.5.2”
Warning message:
“package ‘stringr’ was built under R version 3.5.2”
Warning message:
“package ‘forcats’ was built under R version 3.5.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [7]:
email_data <- read_csv("../input/email_data.csv")
head(email_data)

Parsed with column specification:
cols(
  recency = col_double(),
  history_segment = col_character(),
  history = col_double(),
  mens = col_double(),
  womens = col_double(),
  zip_code = col_character(),
  newbie = col_double(),
  channel = col_character(),
  segment = col_character(),
  visit = col_double(),
  conversion = col_double(),
  spend = col_double()
)



recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0
6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0
7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0
9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0
2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0
6,2) $100 - $200,134.83,0,1,Surburban,0,Phone,Womens E-Mail,1,0,0


## RCTデータの集計と有意差検定

男性向けのメールが配信されたサンプルとメールが配信されなかったサンプルに限定して分析を行う。そのために女性向けのメールが配信されたデータを削除する。

In [4]:
male_df <- email_data %>%
    filter(segment != "Womens E-Mail") %>%
    mutate(treatment = if_else(segment == "Mens E-Mail", 1, 0))

### 集計

介入の有無で集約統計量の比較を行う。

In [6]:
summary_by_segment <- male_df %>%
    group_by(treatment) %>%
    summarise(conversion_rate = mean(conversion),
              spend_mean = mean(spend),
              count = n())
summary_by_segment

treatment,conversion_rate,spend_mean,count
0,0.005726087,0.6527894,21306
1,0.012531093,1.4226165,21307


### 有意差検定

メールが配信されたグループとそうでないグループの間で購買金額に有意差があるかを有意差検定によって検証する。

In [8]:
mens_mail <- male_df %>%
    filter(treatment == 1) %>%
    pull(spend)

no_mail <- male_df %>%
    filter(treatment == 0) %>%
    pull(spend)

rct_ttest = t.test(mens_mail, no_mail, var.equal=TRUE)
rct_ttest


	Two Sample t-test

data:  mens_mail and no_mail
t = 5.3001, df = 42611, p-value = 1.163e-07
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.4851384 1.0545160
sample estimates:
mean of x mean of y 
1.4226165 0.6527894 


## バイアスのあるデータによる効果の検証

メール配信に購買傾向が一定以上あるという条件をつける。

### データの準備

購買傾向が一定以下のデータをランダムに5割削除する。

In [9]:
set.seed(1)

obs_rate_c <- 0.5
obs_rate_t <- 0.5

In [10]:
biased_data <- male_df %>%
    mutate(obs_rate_c = if_else(
        (history > 300) | (recency < 6) | (channel == "Multichannel"), obs_rate_c, 1),
           obs_rate_t = if_else(
        (history > 300) | (recency < 6) | (channel == "Multichannel"), 1, obs_rate_t),
           random_number = runif(n = NROW(male_df))
          ) %>%
    filter((treatment == 0 & random_number < obs_rate_c) | (treatment == 1 & random_number < obs_rate_t))

これによりメールの配信されていないグループでは昨年の購入額が300より高い、最後の購入が3より小さい、接触チャネルが複数ある、のいずれかに該当するデータをランダムに半分削除している。
逆にメールの配信されているグループでは逆に上の条件のいずれにも該当しないデータをランダムに半分削除している。

### データの集計

In [11]:
summary_by_segment_biased <- biased_data %>%
    group_by(treatment) %>%
    summarise(conversion_rate = mean(conversion),
              spend_mean = mean(spend),
              count = n())
summary_by_segment_biased

treatment,conversion_rate,spend_mean,count
0,0.004977838,0.5483062,14665
1,0.013431794,1.5277526,17198


### 有意差検定

In [13]:
mens_mail_biased <- biased_data %>%
    filter(treatment == 1) %>%
    pull(spend)

no_mail_biased <- biased_data %>%
    filter(treatment == 0) %>%
    pull(spend)

ttest_biased <- t.test(mens_mail_biased, no_mail_biased, var.equal=TRUE)
ttest_biased


	Two Sample t-test

data:  mens_mail_biased and no_mail_biased
t = 5.6708, df = 31861, p-value = 1.433e-08
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.6409145 1.3179784
sample estimates:
mean of x mean of y 
1.5277526 0.5483062 


意図的にバイアスを発生させたことにより購買額の差が大きくなっている。また有意差検定のp値もより小さい値になっており効果が過剰に評価されてしまっている。